<a href="https://colab.research.google.com/github/dfatpnuk/bayesian-rl/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pgmpy
import numpy

In [ ]:
from pgmpy.models import BayesianNetwork
G = BayesianNetwork()

In [ ]:
G.add_nodes_from(['a', 'b'])
G.add_edges_from([('a', 'b'), ('b', 'c')])

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
student = BayesianNetwork([('diff', 'grades'), ('aptitude', 'grades')])
grades_cpd = TabularCPD('grades', 3, [[0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.8,0.8,0.8,0.8,0.8,0.8]],
                        evidence=['diff', 'aptitude'], evidence_card=[2, 3],
                        state_names={'grades': ['gradeA', 'gradeB', 'gradeC'],
                                     'diff': ['easy', 'hard'],
                                     'aptitude': ['low', 'medium', 'high']})
student.add_cpds(grades_cpd)

In [ ]:
from pgmpy.models import BayesianNetwork
G = BayesianNetwork()
G.add_nodes_from(['grade', 'intel'])
G.add_edge('grade', 'intel')

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
model = BayesianNetwork([('A', 'B'), ('B', 'C')])
cpd_a = TabularCPD('A', 2, [[0.2], [0.8]])
cpd_b = TabularCPD('B', 2, [[0.3, 0.7], [0.7, 0.3]],
                   evidence=['A'],
                   evidence_card=[2])
cpd_c = TabularCPD('C', 2, [[0.1, 0.9], [0.9, 0.1]],
                   evidence=['B'],
                   evidence_card=[2])
model.add_cpds(cpd_a, cpd_b, cpd_c)
copy_model = model.copy()
copy_model.nodes()
copy_model.edges()
len(copy_model.get_cpds())

3

In [ ]:
from pgmpy.utils import get_example_model
asia = get_example_model('asia')
asia.edges()
do_bronc = asia.do(['bronc'])

In [ ]:
asia.edges()

OutEdgeView([('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')])

In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
data = pd.DataFrame(data={'A': [0, 0, 1], 'B': [0, 1, 0], 'C': [1, 1, 0]})
model = BayesianNetwork([('A', 'C'), ('B', 'C')])
model.fit(data)
model.get_cpds()

[<TabularCPD representing P(A:2) at 0x78142c1572e0>,
 <TabularCPD representing P(C:2 | A:2, B:2) at 0x78142c155990>,
 <TabularCPD representing P(B:2) at 0x78142c155a20>]

Examples taken from: https://pgmpy.org/models/bayesiannetwork.html

In [ ]:
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
model = get_example_model('alarm')
# Generate some new data.
data = BayesianModelSampling(model).forward_sample(int(1e3))
model.fit_update(data)

  0%|          | 0/37 [00:00<?, ?it/s]

Can we apply Reinforcement Learning to Bayesian Network Structure Learning?

Can we apply a greedy hill-climbing approach to this task?